## dataset
Old dataset cleaning code

In [ ]:
# read in dataset
with open(ORIGINAL_TARGET, "r") as f:
    lines = f.readlines()

# remove empty lines and
# write to new .txt file
with open(DATASET_TARGET, "w+") as f:
    lines = filter(str.strip, lines)
    f.writelines(lines)

## project embeddings
Old project embeddings code that I manually implemented for averaging block embeddings to obtain the project embeddings.

In [55]:
# read in the projects text file into df
def csv_to_df(filepath, columns=[]):
    df = pd.read_csv(filepath, sep="\n", header=None)
    df.columns = columns
#     df = df.set_index('project_text')
    return df

df = csv_to_df(train_target, columns=['project_text'])
print(df.shape)
display(df.sample(n=5))

(1000, 1)


,project_text
542,_STARTSTACK_ event_whenthisspriteclicked _NEXT...
492,_STARTSTACK_ event_whenkeypressed _MENU_ menu_...
649,_STARTSTACK_ event_whenflagclicked _NEXT_ cont...
568,_STARTSTACK_ event_whenthisspriteclicked _NEXT...
116,_STARTSTACK_ event_whenflagclicked _NEXT_ cont...


In [92]:
# apply to each row: the split function on the project text on space (" ")
tokens_df = df.apply(lambda row: row['project_text'].split(" "), axis=1)
tokens_df = pd.DataFrame(tokens_df)
tokens_df.columns = ['project_tokens']

In [100]:
print("Number of tokens: ", len(tokens_df.iloc[100]['project_tokens']))

Number of tokens:  1051


In [94]:
def get_batch_word_embeddings(project_tokens):
    # this function gets all word embeddings for tokens in a single project.
    # apply the get_word_vector to each token in project.
    return list(map(model.get_word_vector, project_tokens))
    
# map the get_word_vector function from fasttext to each token from split list above
embeddings_df = tokens_df.apply(lambda row: get_batch_word_embeddings(row['project_tokens']), axis=1)
embeddings_df = pd.DataFrame(embeddings_df)
embeddings_df.columns = ['embeddings']

In [95]:
embeddings_df = embeddings_df.apply(lambda row: np.array(row['embeddings']), axis=1)
embeddings_df = pd.DataFrame(embeddings_df)
embeddings_df.columns = ['embeddings']
display(embeddings_df)

,embeddings
0,"[[0.21450824, -0.045012362, -0.13869916, 0.273..."
1,"[[0.21450824, -0.045012362, -0.13869916, 0.273..."
2,"[[0.21450824, -0.045012362, -0.13869916, 0.273..."
3,"[[0.21450824, -0.045012362, -0.13869916, 0.273..."
4,"[[0.21450824, -0.045012362, -0.13869916, 0.273..."
...,...
995,"[[0.21450824, -0.045012362, -0.13869916, 0.273..."
996,"[[0.21450824, -0.045012362, -0.13869916, 0.273..."
997,"[[0.21450824, -0.045012362, -0.13869916, 0.273..."
998,"[[0.21450824, -0.045012362, -0.13869916, 0.273..."


In [101]:
print(embeddings_df.iloc[100]['embeddings'].shape)

(1051, 128)


In [102]:
# take the mean of those vectors to get the project embedding
project_embeddings = embeddings_df.apply(lambda row: np.mean(row['embeddings'], axis=0), axis=1)
project_embeddings = pd.DataFrame(project_embeddings)
project_embeddings.columns = ['embedding']
display(project_embeddings)

,embedding
0,"[0.023995442, 0.022763226, 0.0007133481, 0.036..."
1,"[-0.016639246, -0.0019163504, -0.0150304735, -..."
2,"[-0.05557948, -0.018262913, -0.03116151, -0.00..."
3,"[0.04061277, 0.031710193, -0.006383933, 0.0316..."
4,"[0.047298573, -0.009121757, -0.043928746, 0.06..."
...,...
995,"[-0.030901695, -0.03857236, -0.032687176, 0.02..."
996,"[0.06905353, 0.053826157, -0.0106923, 0.042574..."
997,"[0.06646415, -0.020444186, 0.011007728, 0.0525..."
998,"[0.08105424, 0.0006001053, -0.015349248, 0.066..."


In [105]:
print(project_embeddings.iloc[0]['embedding'].shape)

(128,)


## sentence vectors
Old code attempting to use fasttext command line utility.

In [ ]:
!ls {"/opt/conda/lib/python3.7/site-packages/"}

In [ ]:
fasttext_target = "/opt/conda/lib/python3.7/site-packages/fasttext"
!{fasttext_target} print-sentence-vectors {model_target} < {train_target}